In [2]:
from torchvision.models.detection import fasterrcnn_resnet50_fpn
import torch

## Faster Rcnn.pt test

In [3]:
NUM_CLASSES = 8  
model = fasterrcnn_resnet50_fpn(pretrained=False, num_classes=NUM_CLASSES)
# Load the state dictionary
model.load_state_dict(torch.load("fasterrcnn_custom_new.pt",map_location=torch.device('cpu')))


c:\Users\SONG\anaconda3\envs\transformer\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\SONG\anaconda3\envs\transformer\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
C:\Users\SONG\AppData\Local\Temp\ipykernel_21944\1743769785.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value fo

<All keys matched successfully>

In [4]:
# Move model to CPU
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# Set the model to evaluation mode
model.eval()

print("Model loaded successfully!")

Model loaded successfully!


need to change json path here 

In [ ]:
import json

json_file = "../data/test/result.json"  
try:
    with open(json_file, "r") as f:
        metadata = json.load(f)
        print(f"Loaded metadata from {json_file}.")
except FileNotFoundError:
    print(f"Error: JSON file not found at {json_file}.")
    raise
except json.JSONDecodeError as e:
    print(f"Error decoding JSON file: {e}")
    raise

Loaded metadata from ../data/test/result.json.


In [6]:
import os
output_dir = "./predictions"
os.makedirs(output_dir, exist_ok=True)
print(f"Output directory set to {output_dir}.")

Output directory set to ./predictions.


In [20]:

import json
import time
from PIL import Image
from torchvision.transforms import functional as F

predictions = []

for idx, image_meta in enumerate(metadata.get("images", [])[:5]):  
    try:
        # Load and preprocess the image
        image_path = os.path.join("../data/test", image_meta["file_name"])
        image = Image.open(image_path).convert("RGB")
        input_tensor = F.to_tensor(image).unsqueeze(0).to(device)

        # Measure inference time
        start_time = time.time()
        outputs = model(input_tensor)
        inference_time = time.time() - start_time

        # Post-process outputs
        for box, label, score in zip(
            outputs[0]['boxes'].tolist(),
            outputs[0]['labels'].tolist(),
            outputs[0]['scores'].tolist()
        ):
            if score > 0.5:  # Confidence threshold
                bbox = [box[0], box[1], box[2] - box[0], box[3] - box[1]]  # Convert to [x, y, w, h]
                predictions.append({
                    "image_id": image_meta["id"],
                    "category_id": int(label),
                    "bbox": [float(coord) for coord in bbox],  # Convert to float
                    "score": float(score),  # Convert to float
                    "inference_time": inference_time  # Store inference time
                })

        print(f"Processed image {idx + 1}: {image_meta['file_name']} (Inference time: {inference_time:.4f} seconds)")

    except Exception as e:
        print(f"Error processing image {image_meta['file_name']}: {e}")

# Save predictions to JSON
output_json = "./predictions/predictions_i5-8265U_fasterrcnn.pt.json"
with open(output_json, "w") as f:
    json.dump(predictions, f, indent=4)
print(f"Predictions saved to {output_json}.")

Processed image 1: images/194e378c-608.png (Inference time: 2.1900 seconds)
Processed image 2: images/fcfa48ab-589.png (Inference time: 2.0834 seconds)
Processed image 3: images/986425fd-590.png (Inference time: 2.2694 seconds)
Processed image 4: images/2d82eb4c-591.png (Inference time: 2.1666 seconds)
Processed image 5: images/9ef8c916-592.png (Inference time: 2.1483 seconds)
Predictions saved to ./predictions/predictions_i5-8265U_fasterrcnn.pt.json.


## ONNX result 

In [22]:
import onnxruntime
# Load the ONNX model once before the loop
onnx_model_path = "fasterrcnn_custom_new.onnx"
try:
    session = onnxruntime.InferenceSession(onnx_model_path)  # Open session once
    print(f"Loaded ONNX model from {onnx_model_path}.")
except Exception as e:
    print(f"Error loading ONNX model: {e}")
    raise


Loaded ONNX model from fasterrcnn_custom_new.onnx.


In [25]:
from torchvision.transforms import functional as F
import time  # Import time module for measuring inference time


# Preprocess and predict on each image
predictions = []
for idx, image_meta in enumerate(metadata.get("images", [])[:5]):  # Process only the first 5 images for testing):
    try:
        # Get image path
        image_path = os.path.join("../data/test", image_meta["file_name"])
        if not os.path.exists(image_path):
            print(f"Warning: Image not found at {image_path}. Skipping.")
            continue

        # Load and preprocess the image
        print(f"Processing image {idx + 1}/{len(metadata['images'])}: {image_meta['file_name']}")
        image = Image.open(image_path).convert("RGB")
        # Resize the image to 224x224
        resized_image = image.resize((224, 224))
        input_tensor = F.to_tensor(resized_image).unsqueeze(0).numpy()

        # Measure inference time
        start_time = time.time()
        ort_inputs = {session.get_inputs()[0].name: input_tensor}
        ort_outs = session.run(None, ort_inputs)
        inference_time = time.time() - start_time

        # Post-process the output
        boxes, labels, scores = ort_outs
        boxes, labels, scores = ort_outs
        for box, label, score in zip(boxes, labels, scores):
            if score > 0.5:  # Apply confidence threshold
                # Convert box format [x1, y1, x2, y2] to [x, y, width, height]
                bbox = [float(box[0]), float(box[1]), float(box[2] - box[0]), float(box[3] - box[1])]
                predictions.append({
                    "image_id": image_meta["id"],
                    "category_id": int(label),  # Convert to int
                    "bbox": bbox,
                    "score": float(score),  # Convert to float
                    "inference_time": float(inference_time)  # Convert to float
                })

        print(f"Processed image {image_meta['file_name']} with {len(predictions)} detections.")
    except Exception as e:
        print(f"Error processing image {image_meta['file_name']}: {e}")
        continue

# Save results to a JSON file
output_json = os.path.join(output_dir, "predictions_i5-8265U_fasterrcnn.onnx.json")
try:
    with open(output_json, "w") as f:
        json.dump(predictions, f, indent=4)
        print(f"Predictions saved to {output_json}.")
except Exception as e:
    print(f"Error saving predictions to {output_json}: {e}")
    raise

Processing image 1/178: images/194e378c-608.png
Processed image images/194e378c-608.png with 2 detections.
Processing image 2/178: images/fcfa48ab-589.png
Processed image images/fcfa48ab-589.png with 2 detections.
Processing image 3/178: images/986425fd-590.png
Processed image images/986425fd-590.png with 4 detections.
Processing image 4/178: images/2d82eb4c-591.png
Processed image images/2d82eb4c-591.png with 4 detections.
Processing image 5/178: images/9ef8c916-592.png
Processed image images/9ef8c916-592.png with 5 detections.
Predictions saved to ./predictions\predictions_i5-8265U_fasterrcnn.onnx.json.


## correct the bounding boxes in JSON file
 back to the dimensions of the original image (640x640)

In [15]:
import json

# Load the JSON file
json_file_path = "./predictions/predictions_i5-8265U_fasterrcnn.onnx.json"
original_width = 640
original_height = 640
resized_width = 224
resized_height = 224

scale_factor_x = original_width / resized_width
scale_factor_y = original_height / resized_height

# Load predictions
with open(json_file_path, "r") as f:
    predictions = json.load(f)

# Correct bounding boxes
for prediction in predictions:
    bbox = prediction["bbox"]  # [x, y, width, height]
    prediction["bbox"] = [
        bbox[0] * scale_factor_x,        # Scale x
        bbox[1] * scale_factor_y,        # Scale y
        bbox[2] * scale_factor_x,        # Scale width
        bbox[3] * scale_factor_y         # Scale height
    ]

# Save corrected predictions back to JSON
output_corrected_json = "./predictions/predictions_i5-8265U_fasterrcnn.onnx.json"
with open(output_corrected_json, "w") as f:
    json.dump(predictions, f, indent=4)

print(f"Corrected predictions saved to {output_corrected_json}.")


Corrected predictions saved to ./predictions/predictions_RTX 3080 TI laptop 95W_fasterrcnn_custom_new.onnx.json.
